In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

import yaml
import pandas as pd
import optuna

from src.data.load import load_data
from src.data.prepare import prepare_data
from src.models.cv_iterator import leave_last_k
from src.features.features import feature_engineering
from src.features.utils import build_rank_input
from src.models.tuner import BayesianSearch
from src.models.ranker import Ranker
from src.models.evaluator import Evaluation, recs_score
from src.models.tracker import launch_mlflow, log_run
from src.models.utils import set_global_seed

In [2]:
# read config
with open('config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

# ensure reproducibility
set_global_seed(seed=config["general"]["seed"])

# set experiment tracking
launch_mlflow()

# set algorithm
ALGORITHM = "XGBRanker"

**Data Preparation & Train/Test Split**

- Load and transform the 3 datasets
- Split whole set into train, validation and test sets by segmenting it temporally

In [3]:
# load and prepare data
dfs = load_data(config=config['data_loader'])
dfs = prepare_data(dataframes=dfs)

In [4]:
# train-test split
df_train, df_test = leave_last_k(df=dfs['data'], config=config['optimization'])
df_train, df_valid = leave_last_k(df=df_train, config=config['optimization'])

**Training Set Enrichment**

- (negative sampling, to be added)
- Feature Engineering: creates cross user-item features for ranking model

In [5]:
# build features for ranking model
user_item_features = feature_engineering(
    dataframes={'user': dfs['user'], 'item': dfs['item'], 'data': df_train}
    )

df_train, df_valid = [
    build_rank_input(ratings=df.iloc[:,:3], features=user_item_features)
    for df in (df_train, df_valid)
    ]

del user_item_features

**Optimization & Evaluation**

- Hyper-parameters - search which hyper-parameters optimize scoring metric for the given algorithm in the validation set
- Evaluation - retrieve best hyper-parameters and recover full training set to evaluate results on test set

In [6]:
# set tuner for hyperparam optimization
tuner = BayesianSearch(
    config["optimization"],
    method="ranker",
    algorithm=ALGORITHM
    )

def objective(trial) -> float:
    return tuner.fit(df_train, df_valid, trial)

# set study
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=config["general"]["seed"])
    )
study.optimize(objective, n_trials= config["optimization"]["n_trials"])

# logging experiment
log_run(experiment_name="Ranker", study=study, tuner=tuner)

[I 2025-07-01 23:53:55,893] A new study created in memory with name: no-name-f70258c3-0904-44bf-afa7-5f922045666e
[I 2025-07-01 23:54:02,140] Trial 0 finished with value: 0.9475287274655356 and parameters: {'learning_rate': 0.1878955193048339, 'gamma': 9.50714306409916, 'max_depth': 12, 'subsample': 0.7993292420985183, 'n_estimators': 104}. Best is trial 0 with value: 0.9475287274655356.
[I 2025-07-01 23:54:18,614] Trial 1 finished with value: 0.9460424920466596 and parameters: {'learning_rate': 0.07884126564776513, 'gamma': 0.5808361216819946, 'max_depth': 14, 'subsample': 0.8005575058716043, 'n_estimators': 298}. Best is trial 0 with value: 0.9475287274655356.
[I 2025-07-01 23:54:23,277] Trial 2 finished with value: 0.9475509544008482 and parameters: {'learning_rate': 0.011271662653605422, 'gamma': 9.699098521619943, 'max_depth': 13, 'subsample': 0.6061695553391381, 'n_estimators': 113}. Best is trial 2 with value: 0.9475509544008482.
[I 2025-07-01 23:54:28,445] Trial 3 finished with

🏃 View run 01JUL2025 at: http://127.0.0.1:5000/#/experiments/859922637182404151/runs/12424e8a637e4194b437e3af7e45f7dc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/859922637182404151
🏃 View run XGBRanker at: http://127.0.0.1:5000/#/experiments/859922637182404151/runs/9ae3d7ebb12e4e87ae922894db06c52e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/859922637182404151


In [7]:
# get anti test-set, i.e., train & validation sets together
df_train = dfs['data'].merge(
    df_test
    , on=['user_id', 'item_id', 'rating'], how='left'
    , indicator=True
    )
df_train = df_train[df_train['_merge'] == 'left_only'].drop(columns=['_merge'])

# create features for ranking model
user_item_features = feature_engineering(
    dataframes={'user': dfs['user'], 'item': dfs['item'], 'data': df_train}
    )

df_train, df_test = [
    build_rank_input(ratings=df.iloc[:,:3], features=user_item_features) for df in (df_train, df_test)
    ]

del user_item_features

In [8]:
# set algorithm best hyperparams
hyperparams = (
    config["optimization"]["ranker"][ALGORITHM]["fixed"]
    | study.best_trial.params
)

# fit model on whole training set
clf = Ranker(algorithm=ALGORITHM, params=hyperparams)
clf.fit(X=df_train["X"], y=df_train["y"], group=df_train["group"])

# test set evaluation
scorer = Evaluation(clf=clf)
scorer.fit(train=tuple(df_train.values()), test=tuple(df_test.values()))

,ndcg
dataset,
train,0.950603
test,0.952903


In [ ]:
# shouldn't be done with test set
# recs_score(df_test.iloc[:, :2], df_train.iloc[:, :3])